In [2]:
import os
from langchain_deepseek import ChatDeepSeek 
from langchain_community.document_loaders import BiliBiliLoader
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import logging

logging.basicConfig(level=logging.INFO)

## 初始化视频数据

{'bvid': 'BV1owrpYKEtP', 'aid': 113769053358965, 'videos': 146, 'tid': 231, 'tid_v2': 2096, 'tname': '', 'tname_v2': '', 'copyright': 1, 'pic': 'http://i2.hdslb.com/bfs/archive/3324ad7f3396ea7a8e363ae05b708ee3d3260436.jpg', 'title': '【官方中英】2025年公认最好的【吴恩达机器学习课程】附课件、代码及实战项目！！！--人工智能/机器学习/深度学习', 'pubdate': 1735979927, 'ctime': 1735979928, 'desc': '视频的所有代码资料、论文资源等，可在gong众号【坚持打代码】回复【22】无偿分享~\n本课程字幕为官方英文字幕资源，由GPT翻译后经人工检查，最大程度上确保字幕适合食用，', 'desc_v2': [{'raw_text': '视频的所有代码资料、论文资源等，可在gong众号【坚持打代码】回复【22】无偿分享~\n本课程字幕为官方英文字幕资源，由GPT翻译后经人工检查，最大程度上确保字幕适合食用，', 'type': 1, 'biz_id': 0}], 'state': 0, 'duration': 83096, 'rights': {'bp': 0, 'elec': 0, 'download': 1, 'movie': 0, 'pay': 0, 'hd5': 0, 'no_reprint': 1, 'autoplay': 1, 'ugc_pay': 0, 'is_cooperation': 0, 'ugc_pay_preview': 0, 'no_background': 0, 'clean_mode': 0, 'is_stein_gate': 0, 'is_360': 0, 'no_share': 0, 'arc_pay': 0, 'free_watch': 0}, 'owner': {'mid': 3493134768016008, 'name': '吴恩达机器学习_AndrewNg', 'face': 'https://i0.hdslb.com/bfs/face/f4f5642f2205cf3a81ba5b5995b79e8229edf95f.jpg'}, 'stat': {'aid': 113769053358965, 'view': 2750513, 'danmaku': 4429, 'reply': 1436, 'favorite': 54032, 'coin': 13097, 'share': 8736, 'now_rank': 0, 'his_rank': 0, 'like': 22361, 'dislike': 0, 'evaluation': '', 'vt': 0}, 'argue_info': {'argue_msg': '', 'argue_type': 0, 'argue_link': ''}, 'dynamic': '', 'cid': 27694664301, 'dimension': {'width': 2160, 'height': 1440, 'rotate': 0}, 'premiere': None, 'teenage_mode': 0, 'is_chargeable_season': False, 'is_story': False, 'is_upower_exclusive': False, 'is_upower_play': False, 'is_upower_preview': False, 'enable_vt': 0, 'vt_display': '', 'is_upower_exclusive_with_qa': False, 'no_cache': False, 'subtitle': {'allow_submit': False, 'list': []}, 'is_season_display': False, 'user_garb': {'url_image_ani_cut': ''}, 'honor_reply': {}, 'like_icon': '', 'need_jump_bv': False, 'disable_show_up_info': False, 'is_story_play': 0, 'is_view_self': False, 'url': 'https://www.bilibili.com/video/BV1owrpYKEtP'}

In [3]:
video_urls = [
    # 官方中英】2025年公认最好的【吴恩达机器学习课程】
    "https://www.bilibili.com/video/BV1owrpYKEtP",  
    # （双语字幕）2026年公认最好的【吴恩达深度学习课程】
    "https://www.bilibili.com/video/BV13HWGzcEh3"
]

try:
    loader= BiliBiliLoader(video_urls=video_urls)
    docs = loader.load()
    print(f"Doc len: {len(docs)}")
except Exception as e:
    print(f"加载BiliBili视频失败: {str(e)}")

Doc len: 2


In [4]:
v1_meta = docs[0].metadata
import copy
v1_meta = copy.deepcopy(v1_meta)
print(f"页面视频的个数: {len(v1_meta['pages'])}")
print(v1_meta['pages'][0])
del v1_meta['pages']
print(v1_meta)

页面视频的个数: 146
{'cid': 27694664301, 'page': 1, 'from': 'vupload', 'part': '1.欢迎来到机器学习（课程配套作业与习题见评论区）', 'duration': 401, 'vid': '', 'weblink': '', 'dimension': {'width': 2160, 'height': 1440, 'rotate': 0}, 'first_frame': 'http://i1.hdslb.com/bfs/storyff/n250104sa25gdh6ovyqolg86q8t4rfrq_firsti.jpg', 'ctime': 1735979928}
{'bvid': 'BV1owrpYKEtP', 'aid': 113769053358965, 'videos': 146, 'tid': 231, 'tid_v2': 2096, 'tname': '', 'tname_v2': '', 'copyright': 1, 'pic': 'http://i2.hdslb.com/bfs/archive/3324ad7f3396ea7a8e363ae05b708ee3d3260436.jpg', 'title': '【官方中英】2025年公认最好的【吴恩达机器学习课程】附课件、代码及实战项目！！！--人工智能/机器学习/深度学习', 'pubdate': 1735979927, 'ctime': 1735979928, 'desc': '视频的所有代码资料、论文资源等，可在gong众号【坚持打代码】回复【22】无偿分享~\n本课程字幕为官方英文字幕资源，由GPT翻译后经人工检查，最大程度上确保字幕适合食用，', 'desc_v2': [{'raw_text': '视频的所有代码资料、论文资源等，可在gong众号【坚持打代码】回复【22】无偿分享~\n本课程字幕为官方英文字幕资源，由GPT翻译后经人工检查，最大程度上确保字幕适合食用，', 'type': 1, 'biz_id': 0}], 'state': 0, 'duration': 83096, 'rights': {'bp': 0, 'elec': 0, 'download': 1, 'movie': 0, 'pay': 0, 'hd5':

In [5]:
bili = []

for doc in docs:
    origin = doc.metadata
    metadata = {
        'title': origin.get('title', '未知标题'),
        'author': origin.get('owner', {}).get('name', '未知作者'),
        'source_id': origin.get('bvid', "未知ID"),
        "view_stat": origin.get('stat', {}).get('view', -1),
        'video_len': origin.get('duration', -1)
    }
    doc.metadata = metadata
    bili.append(doc)

In [6]:
if not bili:
    print("没有成功加载任何视频，程序退出")
    exit()

## 创建向量存储

In [7]:
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5")
vector_store = Chroma.from_documents(bili, embed_model)

/root/miniconda/envs/myrag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-zh-v1.5
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


## 配置元数据字段信息

In [8]:

meta_field_info = [
    AttributeInfo(name="title", description="视频标题", type="string"),
    AttributeInfo(name="author", description="视频作者", type="string"),
    AttributeInfo(name="view_stat", description="视频观看次数", type="integer"),
    AttributeInfo(name="video_len", description="视频长度", type="integer"),
]

## 创建自查询检索器

In [12]:
llm = ChatDeepSeek(
    model="deepseek-chat", 
    temperature=0, 
    api_key=os.getenv("DEEPSEEK_API_KEY")
    )

retriever = SelfQueryRetriever.from_llm(llm=llm, vectorstore=vector_store,
                document_contents="记录视频标题、作者、观看次数、时长的视频元数据",
                metadata_field_info= meta_field_info,
                enable_limit=True, verbose=True)

## 执行查询示例

In [15]:
queries = [
    "所给的视频里视频时长或长度 最短、最小的视频",
    "时长大于600秒的视频"
]

for query in queries:
    print(f"\n--- 查询: '{query}' ---")
    results = retriever.invoke(query)
    if results:
        for doc in results:
            print(f"标题: {doc.metadata.get('title', '未知标题')}")
            print(f"作者: {doc.metadata.get('author', '未知作者')}")
            print(f"观看次数: {doc.metadata.get('view_stat')}")
            print(f"时长: {doc.metadata.get('video_len', )}秒")
            print("="*50)
    else:
        print("未匹配到结果") 

INFO:httpx:HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"



--- 查询: '所给的视频里视频时长或长度 最短、最小的视频' ---


INFO:langchain.retrievers.self_query.base:Generated Query: query=' ' filter=Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='video_len', value=0), Comparison(comparator=<Comparator.LT: 'lt'>, attribute='video_len', value=60)]) limit=1
INFO:httpx:HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


未匹配到结果

--- 查询: '时长大于600秒的视频' ---


INFO:langchain.retrievers.self_query.base:Generated Query: query=' ' filter=Comparison(comparator=<Comparator.GT: 'gt'>, attribute='video_len', value=600) limit=None


标题: 【官方中英】2025年公认最好的【吴恩达机器学习课程】附课件、代码及实战项目！！！--人工智能/机器学习/深度学习
作者: 吴恩达机器学习_AndrewNg
观看次数: 2754366
时长: 83096秒
标题: （双语字幕）2026年公认最好的【吴恩达深度学习课程】deeplearning.ai（coursera）| 附课件、代码及实战项目
作者: 吴恩达机器学习_StanFord
观看次数: 180540
时长: 106435秒


## 思考

为什么本节的代码中查询“时间最短的视频”时，得到的结果是错误的？
- 因为LLM没有生成正确的过滤、没有tok限制： INFO:langchain.retrievers.self_query.base:Generated Query: query=' ' filter=None limit=None
- 或者生成错误的过滤表达式

## 报错1-LLM生成非法Json

```
File ~/miniconda/envs/myrag/lib/python3.12/site-packages/langchain_core/output_parsers/base.py:250, in BaseOutputParser.parse_result(self, result, partial)
    234 @override
    235 def parse_result(self, result: list[Generation], *, partial: bool = False) -> T:
    236     """Parse a list of candidate model Generations into a specific format.
    237 
    238     The return value is parsed from only the first Generation in the result, which
   (...)    248         Structured output.
    249     """
--> 250     return self.parse(result[0].text)

File ~/miniconda/envs/myrag/lib/python3.12/site-packages/langchain/chains/query_constructor/base.py:66, in StructuredQueryOutputParser.parse(self, text)
     62     return StructuredQuery(
     63         **{k: v for k, v in parsed.items() if k in allowed_keys}
     64     )
     65 except Exception as e:
---> 66     raise OutputParserException(
     67         f"Parsing text\n{text}\n raised following error:\n{e}"
     68     )

OutputParserException: Parsing text
```json
{
    "query": "",
    "filter": "gt(\"video_len\", 600)",
    "limit": 
}
```
 raised following error:
Got invalid JSON object. Error: Expecting value: line 5 column 1 (char 73)
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
```

### 问题原因分析

你遇到的错误是因为大语言模型（LLM）生成了**格式不合法的 JSON 字符串**，具体问题出在 `"limit"` 字段后面**缺少了值**：

```json
{
    "query": "",
    "filter": "gt(\"video_len\", 600)",
    "limit":   // ❌ 错误：冒号后面没有值，JSON 语法非法
}
```

JSON 语法要求：**每个键（key）后面必须紧跟一个合法的值**（如字符串、数字、`null`、对象等）。上面的例子中 `"limit":` 后面直接跟了逗号或大括号，没有提供任何值，因此在解析时抛出错误：

```
Expecting value: line 5 column 1 (char 73)
```

这是典型的“模型生成不规范 JSON”问题，尤其在字段可选时，模型容易省略值而非使用 `null`。

---

### ✅ 解决方案（分短期修复和长期优化）

#### 🔧 方案一：短期修复 —— 自动修复 JSON（推荐立即使用）

在将模型输出传给解析器前，用正则表达式自动补全缺失的值：

```python
import re
from langchain.chains.query_constructor.base import StructuredQueryOutputParser
from langchain_core.exceptions import OutputParserException

def fix_invalid_json(text: str) -> str:
    """修复常见 JSON 错误：为缺失值的字段自动补上 null"""
    # 修复 "limit": 后面直接跟逗号或右大括号的情况
    text = re.sub(r'("limit")\s*:\s*([,}])', r'\1: null\2', text)
    # 修复 "filter": 后面缺失值的情况
    text = re.sub(r'("filter")\s*:\s*([,}])', r'\1: null\2', text)
    return text

# 包装原始解析器，增加容错能力
original_parser = StructuredQueryOutputParser.from_components(...)

def robust_parse(text: str):
    try:
        return original_parser.parse(text)  # 先尝试直接解析
    except OutputParserException:
        fixed_text = fix_invalid_json(text)  # 修复后再试
        try:
            return original_parser.parse(fixed_text)
        except Exception as e:
            raise OutputParserException(
                f"修复后仍解析失败。\n原始输出:\n{text}\n\n修复后:\n{fixed_text}"
            ) from e
```

> ✅ 优点：无需改动模型，快速上线，适合生产环境兜底。

---

#### 🧠 方案二：长期优化 —— 改进提示词（Prompt Engineering）

通过更严格的指令，让模型**始终输出合法 JSON**：

```text
你必须输出严格符合 JSON 格式的对象，包含以下三个字段：
- "query": 字符串类型，无内容时用空字符串 ""
- "filter": 字符串或 null，无过滤条件时必须写 null
- "limit": 整数或 null，无限制时必须写 null

⚠️ 重要规则：
1. 所有字段都必须存在，不能省略
2. 不允许出现 "limit": 这种后面没有值的写法
3. 空值必须用 null 表示，不能留空

✅ 正确示例：
{"query": "教程", "filter": "gt(\"duration\", 300)", "limit": 10}
{"query": "", "filter": null, "limit": null}

❌ 错误示例：
{"query": "", "filter": "gt(...)", "limit": }  ← 缺少值，非法！
```

> ✅ 优点：从源头减少错误，提升系统稳定性。

---

#### ⚙️ 方案三：增强防御 —— 使用 Pydantic 校验

结合 Pydantic 模型进行结构校验，增强鲁棒性：

```python
from pydantic import BaseModel, Field
from typing import Optional

class QuerySchema(BaseModel):
    query: str = ""
    filter: Optional[str] = None
    limit: Optional[int] = None

def parse_with_validation(text: str):
    try:
        return QuerySchema.parse_raw(text)  # 自动处理 null 和类型转换
    except Exception:
        # 可结合方案一的修复逻辑
        fixed = re.sub(r'"limit"\s*:\s*([,}])', r'"limit": null\1', text)
        return QuerySchema.parse_raw(fixed)
```

---

### 🔍 为什么模型会生成这种错误？

1. **模型“偷懒”**：当认为某个字段“不需要”时，倾向于直接省略值，而不是写 `null`
2. **训练数据偏差**：部分训练语料中存在不规范 JSON，模型学会了这种坏习惯
3. **JSON 语法严格性**：人类可以容忍 `"limit":` 这种写法，但程序解析器不行

---

### 📌 最佳实践建议

| 措施 | 说明 | 优先级 |
|------|------|--------|
| ✅ **生产环境加修复层** | 用正则自动补 `null`，兜底防护 | ⭐⭐⭐⭐⭐ |
| ✅ **优化提示词** | 明确要求输出合法 JSON + `null` 规范 | ⭐⭐⭐⭐ |
| ✅ **记录修复日志** | 监控模型输出质量，持续改进 | ⭐⭐⭐ |
| ✅ **使用 Pydantic 校验** | 增强类型安全和容错能力 | ⭐⭐⭐ |

> 💡 **经验总结**：永远不要完全信任 LLM 输出的 JSON。在关键系统中，**“提示词优化 + 自动修复 + 结构校验”三重防护**是最稳健的做法。

通过以上方法，你可以彻底解决此类 JSON 解析失败问题，提升 RAG 系统的稳定性和用户体验。